In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import config, generalizable
config.reload_config()

Loading config for ENV_NAME=dev
Reloaded config
Loading config for ENV_NAME=dev
Reloaded config


In [3]:
import torch
torch.manual_seed(1337)
device = generalizable.get_best_torch_device()

### Load data

In [4]:
# raw_text = generalizable.load_text_file('~/datasets/complete_shakespeare.txt')
raw_text = generalizable.load_text_directory('~/datasets/books')
tds = generalizable.encode_text(raw_text, tokenization='trigram')
del raw_text
full_data = torch.tensor(tds.all_tokens, dtype=torch.long, device=device)
print(full_data.shape, full_data.dtype, full_data.device)

Loading *.txt from: /Users/chris/datasets/books
Loaded 63 files, total length: 48810735
Text length: 48810735, total tokens: 16270245, vocab size: 41160
torch.Size([16270245]) torch.int64 mps:0


In [7]:
TRAIN_FRACTION = 0.9

n = int(TRAIN_FRACTION * len(full_data))
train_data = full_data[:n]
val_data = full_data[n:]

### Define the model

In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        # Logits are the raw (non-normalized) predictions that a classification model generates,
        # which are then passed to the normalization function, like softmax.
        # The cross-entropy loss expects raw logits, not the output of a softmax, so that it can apply its own softmax.
        logits = self.token_embedding_table(idx) # (B,T,C)
        # logits is B*T*C, becuase for each batch, we make `Timestep` predictions, and for each prediction we have C classes
        # One character/token predictions for every timestep across the batch sequences
        # print(f"logits.shape: {logits.shape}")

        if targets is None:
            loss = None
        else:
            # Batch, Timestep, Channels
            B, T, C = logits.shape
            
            # cross_entropy expects channels to be the second dimension, so we reshape
            # This view squishes the first two dimensions into one, so that the shape becomes (B*T, C)
            logits = logits.view(B*T, C)
            # print(f"logits.shape after view: {logits.shape}")
            
            # print(f"targets.shape: {targets.shape}")
            targets = targets.view(B*T)
            # print(f"targets.shape after view: {targets.shape}")
            
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # extract only the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx

### Create a WandB run

In [9]:
import wandb
import humanize

LR = 1e-3
BLOCK_SIZE = 8
BATCH_SIZE = 128
NUM_LOSS_ESTIMATE_BATCHES = 400

m = BigramLanguageModel(tds.vocab_size).to(device)
epochs_trained = 0

wandb.init(
    # set the wandb project where this run will be logged
    project="language-models",
    
    # track hyperparameters and run metadata
    config={
        "learning_rate": LR,
        "batch_size": BATCH_SIZE,
        "block_size": BLOCK_SIZE,
        "model": "bigram",
        "tokenization": "bigram",
    }
)

# create a PyTorch optimizer
# Learning rate should usually be 1e-4, but for small networks like this, 1e-3 works
optimizer = torch.optim.AdamW(m.parameters(), lr=LR)

wandb: Currently logged in as: chrisc. Use `wandb login --relogin` to force relogin


In [10]:
def get_batch(dataset):
    return generalizable.get_batch(dataset, BATCH_SIZE, BLOCK_SIZE, device)

def log_wandb_stats(batches_trained):
    train_loss = generalizable.estimate_loss(NUM_LOSS_ESTIMATE_BATCHES, lambda : get_batch(train_data), m)
    val_loss = generalizable.estimate_loss(NUM_LOSS_ESTIMATE_BATCHES, lambda : get_batch(val_data), m)
    wandb.log({"train_loss": train_loss, "val_loss": val_loss}, step=batches_trained)
    return train_loss, val_loss

def print_text_sample():
    idx = torch.zeros((1, 1), dtype=torch.long, device=device)
    tokens = m.generate(idx = idx, max_new_tokens=500)[0].tolist()
    print(tds.decode_token_list_to_string(tokens))

wandb_interval = 400

log_wandb_stats(epochs_trained * BATCH_SIZE)

for _, steps in enumerate(range(20000)):
    # sample a batch of data
    xb, yb = get_batch(train_data)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    epochs_trained += 1
    
    if epochs_trained % wandb_interval == 0:
        train_loss, val_loss = log_wandb_stats(epochs_trained * BATCH_SIZE)
        
        if epochs_trained % (wandb_interval * 4) == 0:
            print(f"step {humanize.intcomma(epochs_trained)}, train_loss: {train_loss}, val_loss: {val_loss}")

print_text_sample()

RuntimeError: MPS backend out of memory (MPS allocated: 26.27 GB, other allocations: 6.47 GB, max allowed: 36.27 GB). Tried to allocate 6.31 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
# idx = torch.zeros((1, 1), dtype=torch.long, device=device)
# tokens = m.generate(idx = idx, max_new_tokens=500)[0].tolist()
# # print(tds.decode_token_list_to_string(tokens))
# print('\n'.join([tds.int_to_str[tok] for tok in tokens]))
# print_text_sample()

In [10]:
# Finish the WandB run
wandb.finish()

loss,█▇▅▄▃▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,2.43489


In [ ]:
with_chars = """
Gofuidrme :
Toun hu: l:
I w herseso thell de de; omole l to.
Aneverkined leoumy e thuftxy, wlveres juront lure omy hesph yo harer core' veneayoro ne trsw'll isurendime ilerve fin?

BRI meso ot ce pr w o can w'losto h prin hif mem! Soums masan IZBELARomit Mu out ikentharn hen rvaiwit, curencer y ve il
UKI I to welilearas be mpure thiks! hatte'd aized'?

tar m anju tathe afedind inghevere Yowhare fofot foling a ge wr'farmfien ar aw bellathy feldin bawd henoslendothecendeang--

d! ond buthet, miz! 
"""